In [15]:
import copy
import cProfile


myFilename = "eupath.txt"

f = open(myFilename, 'r')
inputstr = f.read()

'''
inputstr =  """0 -> 2
     1 -> 3
     2 -> 1
     3 -> 0,4
     6 -> 3,7
     7 -> 8
     8 -> 9
     9 -> 6"""
'''
input_list = [x.strip() for x in inputstr.split("\n")]
adj_dict = {}
global unexplored_nodes
unexplored_nodes = []

# parse input into a dictionary that stores the adjacency list
for my_str in input_list:
    if len(my_str) > 0:
        rec = my_str.split("->")
        key = int(rec[0].strip())
        
        if unexplored_nodes == []:
            unexplored_nodes.append(key)
            
        adj_dict[int(rec[0].strip())] = [int(x.strip()) for x in rec[1].split(",")]

#Finds two semi-balanced nodes in the graph and connects them to allow an Euler cycle to be created.
def completeCycle(graph):
    edge_counts = {}
    out_edge = None
    in_edge = None
    
    for key in graph:
        vals = graph[key]
        
        if edge_counts.get(key) == None:
            edge_counts[key] = [0,0]
        
        edge_counts[key][0] = len(vals)
        for val in vals:
            if edge_counts.get(val) == None:
                edge_counts[val] = [0,1]
            elif edge_counts.get(val)[1] == "":
                edge_counts[val][1] = 1
            else:
                edge_counts[val][1] +=1
                
    for v in edge_counts:
        out_count = edge_counts[v][0]
        in_count = edge_counts[v][1]
        
        if out_count > in_count:
            out_edge = v 
        elif in_count > out_count:
            in_edge = v
            
    if graph.get(in_edge) == None:
        graph[in_edge] = [out_edge]
    else:
        graph[in_edge].append(out_edge)
    global unexplored_nodes 
    unexplored_nodes = [out_edge]        
    return graph

#given a graph and a start node creates and returns a cycle via a random walk 
def createCycle(graph, unexplored_nodes):
    
    my_cycle = []
    cur_node = unexplored_nodes.pop()
    
    #Add first node
    my_cycle.append(cur_node)
    
    while True:
        #Unexplored edges of the current node
        neighbours = graph.get(cur_node)
        
        if neighbours != None:
            #Explore this edge by consuming it
            cur_neighbour = neighbours.pop()
            
            #Add the edge to the cycle
            my_cycle.append(cur_neighbour)
            
            #All edges of the current node have been explored so consume the current node
            if len(neighbours) == 0:
                graph.pop(cur_node)
            else:
                #If this node has mode unexplored edges it becomes the candidate for being a
                #start node for the next cycle
                unexplored_nodes.append(cur_node)
            
            #The neighbour becomes the current node for the next iteration        
            cur_node = cur_neighbour
            
        else:
            #IF this node has no neighbours, can't go any further.
            if len(graph) > 0 and graph.get(cur_node):
                graph.pop(cur_node)
            break
    
    # Return both the created cycle and the start node for the next cycle
    return my_cycle, unexplored_nodes

#Merges two cycles created in separate runs of createCycle() together
def mergeCycles(cycle1, cycle2):
    index_of = cycle1.index(cycle2[0])
    new_cycle = cycle1[:index_of] + cycle2 + cycle1[index_of+1:]
    return new_cycle


def eulerCycle(graph, unexplored_nodes):
    
    #Create the first cycle
    my_cycle, unexplored_nodes = createCycle(graph, unexplored_nodes)
    
    #The graph is consumed as it is explored so having unexplored edges
    #can be detected by testing if the graph is empty
    has_unexplored_edges = len(graph) > 0
    
    while has_unexplored_edges:
        
        #Create the next cycle
        new_cycle, unexplored_nodes = createCycle(graph, unexplored_nodes)
        
        #Merge the new cycle into my_cycle
        my_cycle = mergeCycles(my_cycle, new_cycle)
        
        has_unexplored_edges = len(graph) > 0
    
    #The final euler cycle is returned
    return my_cycle

def doIt():
    graph = completeCycle(adj_dict)
    cycle_list = eulerCycle(graph,unexplored_nodes)[:-1]
    print ("->".join([str(x) for x in cycle_list]))
    
    
    
cProfile.run('doIt()')

1354->1356->914->913->1129->1131->1166->1926->1924->1925->1166->1165->1167->1131->1130->1210->1211->1212->1130->913->1085->1084->1086->1119->1171->2344->2345->2728->2730->2729->2345->2346->1171->1262->1807->1808->1809->2676->2674->2675->1809->1262->1263->1261->1171->1172->1173->1119->1117->1118->1565->1566->1564->1118->1086->913->751->752->449->450->448->163->231->230->421->423->1913->1912->1914->423->959->958->1429->1654->1655->1656->1429->1431->1430->958->960->423->665->664->2182->2183->2184->664->666->423->643->1988->1989->1987->643->644->894->893->892->644->645->423->422->553->554->555->422->2324->2325->2323->422->230->261->260->259->230->229->163->165->971->970->972->1509->2446->2448->2460->2459->2458->2448->2447->1509->1508->1721->1720->1722->1508->1507->972->165->181->183->989->1815->1813->1814->989->988->1810->1812->1811->988->990->183->182->2070->2068->2069->182->165->1772->1773->1771->165->164->9->16->18->957->955->956->1837->1838->1839->956->18->17->880->1614->1613->1612->88

Reference="https://github.com/llevar/rosalind/blob/master/bioinf_algos_pt1/wk5/euler_path.py"